In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.5 MB/s eta 0:00:00


In [39]:
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
# import shap
# import load_boston
from matplotlib import pyplot as plt
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
# boston=load_boston()


In [14]:
boston = pd.read_csv('sample_data/Boston.csv')

# boston = pd.DataFrame(boston.data, columns=boston.feature_names)

In [53]:
boston.head(5)

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [50]:
boston.isnull().sum()
df = boston

In [55]:
y = df['medv']
X2 = df
X = df.drop(['medv', 'Unnamed: 0'],axis=1)
X.head(3)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

In [57]:
train_dataset = cb.Pool(X_train, y_train)
test_dataset = cb.Pool(X_test, y_test)

In [58]:
model = cb.CatBoostRegressor(loss_function='RMSE')

In [59]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)

Выходные данные были обрезаны до нескольких последних строк (5000).
61:	learn: 1.9302535	test: 2.1337971	best: 2.1337971 (61)	total: 144ms	remaining: 321ms
62:	learn: 1.9141415	test: 2.1272391	best: 2.1272391 (62)	total: 146ms	remaining: 318ms
63:	learn: 1.8920256	test: 2.1258090	best: 2.1258090 (63)	total: 148ms	remaining: 315ms
64:	learn: 1.8703215	test: 2.1147848	best: 2.1147848 (64)	total: 151ms	remaining: 313ms
65:	learn: 1.8519417	test: 2.1095553	best: 2.1095553 (65)	total: 153ms	remaining: 311ms
66:	learn: 1.8275234	test: 2.1021215	best: 2.1021215 (66)	total: 156ms	remaining: 310ms
67:	learn: 1.8099435	test: 2.0959270	best: 2.0959270 (67)	total: 159ms	remaining: 308ms
68:	learn: 1.7923768	test: 2.0943540	best: 2.0943540 (68)	total: 161ms	remaining: 306ms
69:	learn: 1.7815164	test: 2.0940778	best: 2.0940778 (69)	total: 163ms	remaining: 303ms
70:	learn: 1.7525400	test: 2.0939138	best: 2.0939138 (70)	total: 165ms	remaining: 300ms
71:	learn: 1.7307144	test: 2.0899461	best: 2.0899461

{'params': {'depth': 6,
  'iterations': 100,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               4

In [60]:
pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

Testing performance
RMSE: 2.83
R2: 0.90


In [62]:
model.save_model('boston.cbm')
model.save_model('boston.json', format='json')